In [1]:
#General Purpose
import pandas as pd
import numpy as np
import string

#Preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

#Classification and Metrics
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multioutput import ClassifierChain
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import PredefinedSplit
from sklearn.metrics import classification_report, f1_score
from sklearn import metrics

%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
clean_reviews = pd.read_csv('/content/drive/MyDrive/Text Mining/PROJECT/modelling/clean_reviews_dataset.csv')

In [4]:
clean_reviews = clean_reviews.dropna(subset=['cleaned_reviews_reduced_words'])
clean_reviews = clean_reviews[clean_reviews['cleaned_reviews_reduced_words'].apply(lambda x: len(str(x)) > 0)]

In [5]:
mlb = MultiLabelBinarizer()
binarised_labels = mlb.fit_transform(clean_reviews['Cuisines'])
clean_reviews['labels'] = binarised_labels.tolist()

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    clean_reviews['cleaned_reviews_reduced_words'], binarised_labels, test_size=0.2, random_state=42, stratify=binarised_labels)

X_train = np.array([str(x) for x in X_train])

In [10]:
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42)

In [13]:
X_train_full = np.concatenate((X_train, X_val), axis=0)
y_train_full = np.concatenate((y_train, y_val), axis=0)

split_index = [-1] * len(X_train) + [0] * len(X_val)
predefined_split = PredefinedSplit(test_fold=split_index)

In [14]:
bigram_tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,1), token_pattern=r"(?u)\b\w+\b")
bigram_bow_vectorizer = CountVectorizer(ngram_range=(1,1), token_pattern=r"(?u)\b\w+\b")

In [15]:
scores = ["accuracy", "precision_weighted", "recall_weighted", "f1_weighted"]

#### One vs rest

In [16]:

from sklearn.naive_bayes import MultinomialNB

pipeline_nb = Pipeline([
    ('vectorizer', bigram_tfidf_vectorizer),
    ('classifier', OneVsRestClassifier(MultinomialNB()))])
param_grid_nb = {
    'vectorizer': [bigram_tfidf_vectorizer, bigram_bow_vectorizer],
    'classifier__estimator__alpha': [0.1, 1, 10]}

grid_search = GridSearchCV(
    estimator=pipeline_nb,
    param_grid=param_grid_nb,
    cv=predefined_split,
    scoring=scores,
    refit="f1_weighted",
    verbose=4)

grid_result = grid_search.fit(X_train_full, y_train_full)

Fitting 1 folds for each of 6 candidates, totalling 6 fits
[CV 1/1] END classifier__estimator__alpha=0.1, vectorizer=TfidfVectorizer(token_pattern='(?u)\\b\\w+\\b'); accuracy: (test=0.175) f1_weighted: (test=0.851) precision_weighted: (test=0.891) recall_weighted: (test=0.866) total time=   1.1s
[CV 1/1] END classifier__estimator__alpha=0.1, vectorizer=CountVectorizer(token_pattern='(?u)\\b\\w+\\b'); accuracy: (test=0.202) f1_weighted: (test=0.877) precision_weighted: (test=0.876) recall_weighted: (test=0.882) total time=   1.1s
[CV 1/1] END classifier__estimator__alpha=1, vectorizer=TfidfVectorizer(token_pattern='(?u)\\b\\w+\\b'); accuracy: (test=0.070) f1_weighted: (test=0.784) precision_weighted: (test=0.860) recall_weighted: (test=0.833) total time=   1.0s
[CV 1/1] END classifier__estimator__alpha=1, vectorizer=CountVectorizer(token_pattern='(?u)\\b\\w+\\b'); accuracy: (test=0.169) f1_weighted: (test=0.866) precision_weighted: (test=0.883) recall_weighted: (test=0.876) total time= 

In [19]:
print("Best Parameters (Naive Bayes One Vs Rest): ", grid_result.best_params_)

Best Parameters (Naive Bayes One Vs Rest):  {'classifier__estimator__alpha': 0.1, 'vectorizer': CountVectorizer(token_pattern='(?u)\\b\\w+\\b')}


In [20]:
for score in scores:
    mean_score = grid_result.cv_results_[f'mean_test_{score}'][grid_result.best_index_]
    print(f"Naive Bayes One Vs Rest: \n{score} = {round(mean_score, 3)}")

Naive Bayes One Vs Rest: 
accuracy = 0.202
Naive Bayes One Vs Rest: 
precision_weighted = 0.876
Naive Bayes One Vs Rest: 
recall_weighted = 0.882
Naive Bayes One Vs Rest: 
f1_weighted = 0.877


#### Classifier Chain

In [21]:
pipeline_nb_cchain = Pipeline([
    ('vectorizer', bigram_tfidf_vectorizer),
    ('classifier', ClassifierChain(MultinomialNB()))])


param_grid_cc = {'vectorizer': [bigram_tfidf_vectorizer, bigram_bow_vectorizer],
    'classifier__order': [None, 'random'],
    'classifier__base_estimator__alpha': [0.1, 1, 10] }


grid_search = GridSearchCV(
    estimator=pipeline_nb_cchain,
    param_grid=param_grid_cc,
    cv=predefined_split,
    scoring=scores,
    refit="f1_weighted",
    verbose=4)

grid_result = grid_search.fit(X_train_full, y_train_full)

Fitting 1 folds for each of 12 candidates, totalling 12 fits
[CV 1/1] END classifier__base_estimator__alpha=0.1, classifier__order=None, vectorizer=TfidfVectorizer(token_pattern='(?u)\\b\\w+\\b'); accuracy: (test=0.175) f1_weighted: (test=0.854) precision_weighted: (test=0.877) recall_weighted: (test=0.873) total time=   1.5s
[CV 1/1] END classifier__base_estimator__alpha=0.1, classifier__order=None, vectorizer=CountVectorizer(token_pattern='(?u)\\b\\w+\\b'); accuracy: (test=0.214) f1_weighted: (test=0.877) precision_weighted: (test=0.870) recall_weighted: (test=0.888) total time=   1.0s
[CV 1/1] END classifier__base_estimator__alpha=0.1, classifier__order=random, vectorizer=TfidfVectorizer(token_pattern='(?u)\\b\\w+\\b'); accuracy: (test=0.181) f1_weighted: (test=0.852) precision_weighted: (test=0.880) recall_weighted: (test=0.875) total time=   1.0s
[CV 1/1] END classifier__base_estimator__alpha=0.1, classifier__order=random, vectorizer=CountVectorizer(token_pattern='(?u)\\b\\w+\\b')

In [22]:
print("Best Parameters (Naive Bayes CChain:): ", grid_result.best_params_)

Best Parameters (Naive Bayes CChain:):  {'classifier__base_estimator__alpha': 0.1, 'classifier__order': 'random', 'vectorizer': CountVectorizer(token_pattern='(?u)\\b\\w+\\b')}


In [23]:
for score in scores:
    mean_score = grid_result.cv_results_[f'mean_test_{score}'][grid_result.best_index_]
    print(f"Naive Bayes CChain: \n{score} = {round(mean_score, 3)}")

Naive Bayes CChain: 
accuracy = 0.217
Naive Bayes CChain: 
precision_weighted = 0.87
Naive Bayes CChain: 
recall_weighted = 0.89
Naive Bayes CChain: 
f1_weighted = 0.877
